In [37]:
# Imports
from IPython.display import Video
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time
import queue
import statistics

In [38]:
def preview_video(input_video_path: str, output_video_path: str):
    cap = cv2.VideoCapture(input_video_path)
    out = cv2.VideoWriter(output_video_path, -1, 20.0, (640,480))
    # get total number of frames
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    suc, prev = cap.read()
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    out.write(prevgray-prevgray)

    for image in range(int(totalFrames) - 1):

        suc, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   
        
        fps = str(image) 
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        cv2.putText(gray, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
        
        gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        
        out.write(gray_bgr)

        
    cap.release()
    out.release()
    
    
preview_video('gmm_fingers.mp4','gmm_fingers_frames.mp4')
Video('gmm_fingers_frames.mp4')

In [46]:
cap = cv2.VideoCapture('gmm_fingers.mp4')

# get total number of frames
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)


suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

images = np.zeros((np.shape(prevgray)[0]*np.shape(prevgray)[1],int(totalFrames)),np.uint8)

images[:,0] = prevgray.flatten()

for image in range(int(totalFrames) - 1):

    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   
    
    images[:,image] = gray.flatten()
    
    
cap.release()

In [40]:
def show_img_in_notebook(frame):
    RGB_im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    plt.imshow(RGB_im)
    plt.title("Video")
    plt.show()
    
# show_img_in_notebook(images[:,:,100])

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [47]:
labels = np.zeros((int(totalFrames)),np.uint8)

labels[:] = -1
labels[:80] = 5
labels[100:145] = 4
labels[150:210] = 3
labels[220:280] = 2
labels[300:360] = 1
labels[370:420] = 0

In [57]:
test = np.reshape(labels, (int(totalFrames),1))
test_2 = np.transpose(images)

# data= np.concatenate((test,test_2),axis=1)
# labels[labels[]]